# Python code to parse nexus base class nxdl files to python dict and create owl ontology from dict
# Note this is not the ontology to use, just an illustration

# To do:
# NeXus fields as datatype properties
# remove nexus field properties (done) and replace with annotations
# example skos:broader (application definitions)
onto.metadata.comment.append("my first comment")
use .name attribute and give iri for class name?
namespace = onto.get_namespace("http://test.org/onto/pharmaco")


## Notes:
In base classes, the only new fields introduced into included bases classes are in NXdata. These can probably be ignored but if required then include later just for NXdata. Ignore any fields that are not in top-level group.

using ':' or '#' in data propery name seems to fail when asserting with individual. '-','.','_' are ok
Maybe %, $...?


## add cites_group to all classes (base classes & defs) - did we do this before?
## add has_field to all classes? Is this possible? Only want to top level ones either list of strings or link only to unique fields ????????

# == how to write subgroup of NXfield some datatype? ==== test
# == is it possible to use different datatypes (NXfloa etc)??
# owlready2 github: declare_datatype is under development ... is it implemented yet?

## Notes on the ontology:



In [1]:
#get a list of NeXus base class urls from github

#pip install pygithub
token = "e9bedb765c37956e75c9a552f93455034ce54aa5"

out_path = '/home/spc93/ontology/'
onto_iri = 'http://test.org/NeXus_onto5.owl'

from github import Github
import datetime
import types

g = Github(token)
repo = g.get_repo('nexusformat/definitions')

base_class_url = []
for file in repo.get_contents("base_classes"):
    if str(file).split('.')[-2] == 'nxdl':
        base_class_url += [file.download_url]

#get a list of NeXus base application definition urls from github
        
application_url = []
for file in repo.get_contents("applications"):
    try:
        if str(file).split('.')[-2] == 'nxdl':
            application_url += [file.download_url]
    except:
        pass
    
#use closest owlready2 data types
type_convert = {
    'NX_CHAR':str,
    'NX_FLOAT':float,
    'NX_INT':int,
    'NX_UINT':int,
    'NX_POSINT':int,
    'NX_NUMBER':float,
    'NX_DATE_TIME':datetime.datetime,
    'NX_BOOLEAN':bool,
    'NX_BINARY':int,
    'ISO8601':datetime.datetime
}  


#join_string = ':'      #string added between joined names
join_string = '-'      #string added between joined names

In [2]:
# parse nexus base class files via url to python dictionary

import xml.dom.minidom
import os
#import yaml
import urllib
import time
from pprint import pprint

_maxTries = 10 # try to parse file this many times before giving up

def addFieldToDict(classDict, field): # make a function to be reused later

    field_name = field.getAttribute('name')

    
    deprecationAttribute = field.getAttribute('deprecated')
    if not deprecationAttribute == '':
        print("=== Deprecation warning %s in %s: %s" % (field_name, className, deprecationAttribute))    

    long_name = className + join_string + field_name

    if not long_name in classDict[className]['fields'].keys():
        classDict[className]['fields'][long_name] = {} # create dictionary for field if doesn't exist

    classDict[className]['fields'][long_name]['fieldName'] = field_name
    classDict[className]['fields'][long_name]['units'] = field.getAttribute('units')
    classDict[className]['fields'][long_name]['xml_file'] = file #xml file where field is defined

    _type = field.getAttribute('type')
    if _type == '':
        _type = 'NX_CHAR'   # default if not specified

    classDict[className]['fields'][long_name]['type'] = _type

    try:
        field_doc = field.getElementsByTagName('doc')[0].firstChild.nodeValue.replace('\n','')
    except:
        field_doc = ''
    classDict[className]['fields'][long_name]['fieldDoc'] = field_doc


classDict = {} # create empty classDict dictionary

#for file in file_list:
for file in base_class_url:
    print(file)

    for i in range(_maxTries):
        try:
            dom1 = xml.dom.minidom.parse(urllib.request.urlopen(file))
            break
        except:
            print('=== Problem parsing %s; try %i times then give up' % (file, _maxTries))
            time.sleep(1)
    
    defn = dom1.getElementsByTagName('definition')[0]
    
    className = defn.getAttribute('name') #class name from name attribute in definition
    
    if not className in classDict.keys():
        classDict[className] = {} # each class is a dictionary - create a new one if doesn't exist
    
    classDict[className]['xml_file'] = file

    #category = defn.getAttribute('category')
    classDict[className]['extends'] = defn.getAttribute('extends')
    classDict[className]['classDoc'] = dom1.getElementsByTagName('doc')[0].firstChild.nodeValue.replace('\n','')
    
    
    if not 'fields' in classDict[className].keys():
        classDict[className]['fields'] = {} # create fields dictionary for class if doesn't exist
    
 
    # look for fields in group but not recursive
    flds = (field for field in defn.getElementsByTagName('field') if field.parentNode == defn)        
    
    for field in flds:
        addFieldToDict(classDict, field)

    

pprint(classDict)



https://raw.githubusercontent.com/nexusformat/definitions/main/base_classes/NXaperture.nxdl.xml
https://raw.githubusercontent.com/nexusformat/definitions/main/base_classes/NXattenuator.nxdl.xml
https://raw.githubusercontent.com/nexusformat/definitions/main/base_classes/NXbeam.nxdl.xml
https://raw.githubusercontent.com/nexusformat/definitions/main/base_classes/NXbeam_stop.nxdl.xml
https://raw.githubusercontent.com/nexusformat/definitions/main/base_classes/NXbending_magnet.nxdl.xml
https://raw.githubusercontent.com/nexusformat/definitions/main/base_classes/NXcapillary.nxdl.xml
https://raw.githubusercontent.com/nexusformat/definitions/main/base_classes/NXcite.nxdl.xml
https://raw.githubusercontent.com/nexusformat/definitions/main/base_classes/NXcollection.nxdl.xml
https://raw.githubusercontent.com/nexusformat/definitions/main/base_classes/NXcollimator.nxdl.xml
https://raw.githubusercontent.com/nexusformat/definitions/main/base_classes/NXcrystal.nxdl.xml
https://raw.githubusercontent.com/n

                                                                   'scalar all '
                                                                   'pixels are '
                                                                   'the same '
                                                                   'size',
                                                       'fieldName': 'y_pixel_size',
                                                       'type': 'NX_FLOAT',
                                                       'units': 'NX_LENGTH',
                                                       'xml_file': 'https://raw.githubusercontent.com/nexusformat/definitions/main/base_classes/NXdetector.nxdl.xml'},
                           'NXdetector-z_pixel_offset': {'fieldDoc': '\tOffset '
                                                                     'from the '
                                                                     'detector '
                                      

                         'NXmirror-bend_angle_y': {'fieldDoc': '',
                                                   'fieldName': 'bend_angle_y',
                                                   'type': 'NX_FLOAT',
                                                   'units': 'NX_ANGLE',
                                                   'xml_file': 'https://raw.githubusercontent.com/nexusformat/definitions/main/base_classes/NXmirror.nxdl.xml'},
                         'NXmirror-coating_material': {'fieldDoc': '',
                                                       'fieldName': 'coating_material',
                                                       'type': 'NX_CHAR',
                                                       'units': '',
                                                       'xml_file': 'https://raw.githubusercontent.com/nexusformat/definitions/main/base_classes/NXmirror.nxdl.xml'},
                         'NXmirror-coating_roughness': {'fieldDoc': '',
           

                                   'NXsample_component-unit_cell_abc': {'fieldDoc': 'Crystallography '
                                                                                    'unit '
                                                                                    'cell '
                                                                                    'parameters '
                                                                                    'a, '
                                                                                    'b, '
                                                                                    'and '
                                                                                    'c',
                                                                        'fieldName': 'unit_cell_abc',
                                                                        'type': 'NX_FLOAT',
                                                             

In [3]:
# parse nexus application definitions
# extract extra base class fields and add to base class dictionary

# cntrl / to comment/uncomment
# tab/shift_tab to block tab/untab


import xml.dom.minidom
import os
#import yaml
import urllib
import time

#file = application_url[4]

applicationDict = {}

for file in application_url:


    dom1 = xml.dom.minidom.parse(urllib.request.urlopen(file)) # pick one at random 
    classList = ''
    group = dom1.getElementsByTagName('group')

    classList, classNameList = [], []
    for defn in group:
        className = defn.getAttribute('type')
        classList += [defn]
        classNameList += [className]
    
        # look for fields in group but not recursive
        flds = (field for field in defn.getElementsByTagName('field') if field.parentNode == defn)           
        for field in flds:
            addFieldToDict(classDict, field)

    for docelement in dom1.getElementsByTagName('doc'):
        docstr = ''
        if docelement.parentNode.tagName == 'definition':
            docstr = docelement.firstChild.nodeValue.replace('\n','')
            break
            
    
    # get informat ion about application definition (name, xml_file, extends, doc) and add to dict
    defn = dom1.getElementsByTagName('definition')[0] 
    defn_name = defn.getAttribute('name')
    applicationDict[defn_name] = {} # new entry with definition nae as key
    applicationDict[defn_name]['extends'] = defn.getAttribute('extends')
    #applicationDict[defn_name]['doc'] = dom1.getElementsByTagName('doc')[0].firstChild.nodeValue.replace('\n','')
    applicationDict[defn_name]['doc'] = docstr
    applicationDict[defn_name]['xml_file'] = file
    applicationDict[defn_name]['groups_cited'] = classNameList
    applicationDict[defn_name]['base_classes'] = classList
    


#pprint(classDict)
pprint(applicationDict)
        
        
        

=== Deprecation warning radiation in NXsource: Use either (or both) ``probe`` or ``type`` fields from ``NXsource`` (issue #765)
=== Deprecation warning incident_wavelength_weight in NXbeam: use incident_wavelength_weights, see https://github.com/nexusformat/definitions/issues/837
{'NXarchive': {'base_classes': [<DOM Element: group at 0x7fc5a597d550>,
                                <DOM Element: group at 0x7fc5a5977b80>,
                                <DOM Element: group at 0x7fc5a5977160>,
                                <DOM Element: group at 0x7fc5a59773a0>,
                                <DOM Element: group at 0x7fc5a5a0c280>],
               'doc': '\tThis is a definition for data to be archived by ICAT '
                      '(http://www.icatproject.org/).\t\t.. text from the '
                      'icatproject.org site\t\t        the database (with '
                      'supporting software) that provides an \t        '
                      'interface to all ISIS experime

In [4]:
# messy attempt to create something that looks like NeXus data types
# re-write later
# remove c individuals

from owlready2 import *
import copy

onto_path.append(out_path)
onto = get_ontology(onto_iri)

class Hex(object):
    def __init__(self, value):
        self.value = value
   
def parser(s):
    return Hex(int(s, 16))
   
def unparser(x):
    h = hex(x.value)[2:]
    if len(h) % 2 != 0: return "0%s" % h
    return h

typedict = {} #dictionary of new types created in owlready2 (key = nx type string, value = new type, needed later)

for _nxtype in type_convert.keys():

    declare_datatype(Hex, '%s#%s' % (onto_iri, _nxtype), parser, unparser)

    typedict[_nxtype] = copy.deepcopy(Hex)
        
    with onto:
        class p(Thing >> Hex): pass
        class C(Thing): pass
        c1 = C()
        c1.p.append(Hex(14))

    onto.save()

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [5]:
typedict

{'NX_CHAR': __main__.Hex,
 'NX_FLOAT': __main__.Hex,
 'NX_INT': __main__.Hex,
 'NX_UINT': __main__.Hex,
 'NX_POSINT': __main__.Hex,
 'NX_NUMBER': __main__.Hex,
 'NX_DATE_TIME': __main__.Hex,
 'NX_BOOLEAN': __main__.Hex,
 'NX_BINARY': __main__.Hex,
 'ISO8601': __main__.Hex}

In [6]:
a = copy.deepcopy(Hex)

In [7]:
type(a)

type

In [8]:
#pip install owlready2

from owlready2 import *

onto_path.append(out_path)
onto = get_ontology(onto_iri)

with onto:

    ## Classes (generic)
    
    #declare_datatype('NXfloet','https://github.com/nexusformat/definitions/blob/main/nxdlTypes.xsd','','')

    class NeXus(Thing):
        comment = 'NeXus concept bla bla'
        
    class dataset(Thing):
        comment = 'Dummy data sets'
    
    #class NeXusField(NeXus):
    #    comment = 'NeXus field. Unique names are created by prepending the NeXus class name to the NeXus field name'
    
    #class testDataProperty(DataProperty): #################################### delete
    #    range = [float]
    #    comment = 'a test'
    
    class NeXusField(DataProperty, FunctionalProperty):
        comment = 'NeXus field (DataProperty). Unique names are created by prepending the NeXus class name to the NeXus field name'
    
    class NXobject(NeXus):
        comment = 'NXobject'
    
    class NeXusBaseClass(NXobject):
        comment = 'NeXusBaseClass'
    
    class NeXusApplicationDefinition(NXobject):
        comment = 'NeXus Application Definition'
      
    #class NeXusUnitCategory(NeXus):
    #    comment = 'NeXus Unit Category'
        
    #class NeXusUnit(NeXusUnitCategory):
    #    comment = 'NeXus Unit'
        
    ## properties (generic)
    
    #class hasField(NXobject >> NeXusField):
    #    comment = 'NXobject has NeXus field relationship'
    
    ## is there a shorthand form with inverse?
    ## how to assert the reverse inverse? By reasoner?
    
    #class isFieldOf(ObjectProperty):
    #    domain           = [NeXusField]
    #    range            = [NXobject]
    #    inverse_property = hasField
        
    # TypeError: unsupported operand type(s) for >>: 'DataPropertyClass' and 'ThingClass'
    #class hasUnit(NeXusField >> NeXusUnit):
    #    comment = 'NeXus Field has Unit relationship'
        
    class citesGroup(NXobject >> NeXusBaseClass):
        comment = 'NXobject cites base class relationship'
        
    #TypeError: unsupported operand type(s) for >>: 'DataPropertyClass' and 'type'
    #class hasFieldName(NeXusField >> str):
    #    comment = 'NeXus field name of NeXus field. NeXus field names are unique only within the NeXus class in which they are defined.'
    
    class NXDLurl(AnnotationProperty):
        pass
    
    class NeXusType(AnnotationProperty):
        pass
    
    class unit(AnnotationProperty):
        pass 
    
    ## relationships (generic)

    ## from NXDL files
    
    for nxBaseClass in classDict.keys():
        
        if not nxBaseClass == 'NXobject':    # NXobject can't be subclass of NXobject
            _nx_class = types.new_class(nxBaseClass, (NeXusBaseClass,))
            _nx_class.comment.append(classDict[nxBaseClass]['classDoc'])
            _nx_class.NXDLurl.append(classDict[nxBaseClass]['xml_file'])
           
                
            domn = dataset
            for nxField in classDict[nxBaseClass]['fields'].keys():
                rng = type_convert[classDict[nxBaseClass]['fields'][nxField]['type']] #convert NeXus type to Owlready2 type
                
                _nx_field = types.new_class(nxField, (NeXusField,))
                _nx_field.range = [rng]
                
                _nx_field.comment.append(classDict[nxBaseClass]['fields'][nxField]['fieldDoc'])
      
                _nx_field.NXDLurl.append(classDict[nxBaseClass]['fields'][nxField]['xml_file'])
            
                _nx_field.NeXusType.append(classDict[nxBaseClass]['fields'][nxField]['type'])
                
                if classDict[nxBaseClass]['fields'][nxField]['units'] != '':
                    _nx_field.unit.append(classDict[nxBaseClass]['fields'][nxField]['units'])
                    
                #_nx_class.is_a.append(_nx_field.some(int))   ## this works and gives xds:integer
                #_nx_class.is_a.append(_nx_field.some(Hex)) # this works but always same type
                _nx_class.is_a.append(_nx_field.some(typedict[classDict[nxBaseClass]['fields'][nxField]['type']])) # 
                
            
                #print('Type: %s\t Units: %s' % (classDict[nxBaseClass]['fields'][nxField]['type'],
                #     classDict[nxBaseClass]['fields'][nxField]['units']))
                
                #_nx_class.is_a.append(hasField.some(_nx_field))
                
                
    for application in applicationDict.keys():
        _nx_app = types.new_class(application, (NeXusApplicationDefinition,))
        _nx_app.comment.append(applicationDict[application]['doc'])
        _nx_app.NXDLurl.append(applicationDict[application]['xml_file'])
        for base_class in applicationDict[application]['base_classes']:
            print(base_class)
            #_nx_app.is_a.append(citesGroup.some(base_class)) ##################### fix this
        #_nx_class.is_a.append(hasField.some(_nx_field)) # from before .. but now we have datatype properties - check in protege
        #MyClass.is_a.append(my_property.some(Value))
        #my_drug.has_for_ingredient.append(codeine)
        
onto.save()





<DOM Element: group at 0x7fc5a597d550>
<DOM Element: group at 0x7fc5a5977b80>
<DOM Element: group at 0x7fc5a5977160>
<DOM Element: group at 0x7fc5a59773a0>
<DOM Element: group at 0x7fc5a5a0c280>
<DOM Element: group at 0x7fc5a5a95ee0>
<DOM Element: group at 0x7fc5a5a95dc0>
<DOM Element: group at 0x7fc5a5a95550>
<DOM Element: group at 0x7fc5a5a78160>
<DOM Element: group at 0x7fc5a5a78310>
<DOM Element: group at 0x7fc5a59ec8b0>
<DOM Element: group at 0x7fc5a59ec670>
<DOM Element: group at 0x7fc5a59ec3a0>
<DOM Element: group at 0x7fc5a5922820>
<DOM Element: group at 0x7fc5a5978550>
<DOM Element: group at 0x7fc5a5978160>
<DOM Element: group at 0x7fc5a5978040>
<DOM Element: group at 0x7fc5a5938310>
<DOM Element: group at 0x7fc5a58c3dc0>
<DOM Element: group at 0x7fc5a5998310>
<DOM Element: group at 0x7fc5a5a86940>
<DOM Element: group at 0x7fc5a5a8b160>
<DOM Element: group at 0x7fc5a5a8b280>
<DOM Element: group at 0x7fc5a5a8b670>
<DOM Element: group at 0x7fc5a5a8baf0>
<DOM Element: group at 0x

In [9]:
copy

<module 'copy' from '/dls_sw/apps/python/anaconda/4.6.14/64/envs/python3.8/lib/python3.8/copy.py'>

In [10]:
int

int

# test new datatype

class Hex(object):
    def __init__(self, value):
        self.value = value
   
def parser(s):
    return Hex(int(s, 16))
   
def unparser(x):
    h = hex(x.value)[2:]
    if len(h) % 2 != 0: return "0%s" % h
    return h


#declare_datatype(Hex, "http://www.w3.org/2001/XMLSchema#hexBinary", parser, unparser)
declare_datatype(Hex, 'http://test.org/NeXus_onto5.owl#newtype33', parser, unparser)

#'http://test.org/NeXus_onto5.owl'


#onto = world.get_ontology("http://www.test.org/t.owl")
   
with onto:
    class p(Thing >> Hex): pass
    class C(Thing): pass
    c1 = C()
    c1.p.append(Hex(14))
    
onto.save()
    

# test new datatype - test last bit

class Hex(object):
    def __init__(self, value):
        self.value = value

def parser(x):
    pass

def unparser(x):
    return ''

d = Thing()

#declare_datatype(Hex, "http://www.w3.org/2001/XMLSchema#hexBinary", parser, unparser)
declare_datatype(Hex, 'http://test.org/NeXus_onto5.owl#newtype29', parser, unparser)
#declare_datatype(Hex, 'http://test.org/NeXus_onto5.owl#newtype11', pars, pars)# Error binding parameter 3 - probably unsupported type.


#'http://test.org/NeXus_onto5.owl'


#onto = world.get_ontology("http://www.test.org/t.owl")
   
with onto:
    #class p(Thing >> Hex): pass
    #class C(Thing): pass
    #c1 = C()
    #c1.p.append(Hex(14))
    class p(Thing >> Hex): pass
    class C(Thing): pass
    c1 = C()
    c1.p.append(Hex(14))
    
onto.save()
    

In [11]:
# create individuals for testing


with onto:

#     class dataset(Thing):
#         comment = 'Dataset individuals'

    #problems:
    # two different dataset classes, one from Thing, the other not but with individuals
    # try removing creation of individuals...now just one (from thing)
    # put back ... double classes back... is this really a problem?
    # data property value appears in annotation
    # does dataset1 have an nxfield attribute if no value set? - no
    # doesn't seem to be using the data property properly 
    # non_existent_property doesn't show up even in annotation... but shows up in python attribute
    # move code to previous block - no change
    #
    
    #dataset1 = dataset(NeXusField = [44])
    #dataset1.NeXusField = 44

    
#     class nxfield(DataProperty, FunctionalProperty):
#         domain    = [dataset]
#         range     = [float]
        
    dataset1 = dataset('dataset1')
    setattr(dataset1, 'NXbeam%sfinal_energy' % join_string, 6.6)
    
    dataset2 = dataset('dataset2')
    setattr(dataset2, 'NXattenuator%sthickness' %join_string, 0.3)
        
onto.save()


# >>> with onto:
# ...     class has_for_cost(DataProperty, FunctionalProperty): # Each drug has a single cost
# ...         domain    = [Drug]
# ...         range     = [float]

# >>> my_drug.has_for_cost = 4.2

# >>> print(my_drug.has_for_cost)
# 4.2


#>>> a_blood_based_drug = Drug()
#>>> a_blood_based_drug.is_a.append(BloodBasedProduct)

In [12]:
type(Hex)
type(object)
object.__module__
object()

In [13]:
nxField

'NXxraylens-gas_pressure'

In [14]:
applicationDict['NXarpes']

{'extends': 'NXobject',
 'doc': '    This is an application definition for angular resolved photo electron spectroscopy.        It has been drawn up with hemispherical electron analysers in mind.   ',
 'xml_file': 'https://raw.githubusercontent.com/nexusformat/definitions/main/applications/NXarpes.nxdl.xml',
 'groups_cited': ['NXentry',
  'NXinstrument',
  'NXsource',
  'NXmonochromator',
  'NXdetector',
  'NXsample',
  'NXdata'],
 'base_classes': [<DOM Element: group at 0x7fc5a5a95ee0>,
  <DOM Element: group at 0x7fc5a5a95dc0>,
  <DOM Element: group at 0x7fc5a5a95550>,
  <DOM Element: group at 0x7fc5a5a78160>,
  <DOM Element: group at 0x7fc5a5a78310>,
  <DOM Element: group at 0x7fc5a59ec8b0>,
  <DOM Element: group at 0x7fc5a59ec670>]}

In [15]:
_nx_app.citesGroup

[]

In [16]:
declare_datatype('NXfloet','http://test.org/','','')

153

In [17]:
onto

get_ontology("http://test.org/NeXus_onto5.owl#")